# PC2 - Grupo 2 (Julia)

#### Integrantes
- GARCIA RODRIGUEZ, EMILIO ALONSO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SALAMANCA FERNANDEZ, LUCAS PABLO
- SILVA ANDUJAR, NICOLAS

## 1. Loading and processing the data

In [92]:
using Pkg
using CSV
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings
using Lasso
# using MLBase
using Statistics
using GLMNet
using MLJ

1. Load de data

In [93]:
using CSV

data = CSV.read("wage2015_subsample_inference.csv", DataFrame, types = Dict(:occ2 => String, :ind2 => String))


Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Float64,String
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,30,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,43,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,44,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,47,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


2. Generate de Extra-Flexible Model. This model contains all two-way interactions between the experience polynomials and the indicator variables. 

In [94]:
# Extra-flexible model
eflex_formula = @formula(lwage ~ 1 + sex + (exp1 + exp2 + exp3 + exp4) * (hsg + scl + clg + ad + so + we + ne + occ2 + ind2) + (hsg + scl + clg + ad) * (so + we + ne + occ2 + ind2) + (so + we + ne) * (occ2 + ind2) + occ2 * ind2)
X_eflex = modelmatrix(eflex_formula, data);

println(size(X_eflex))

(5150, 965)


2.1 Generate the array for the outcome variable $Y$ and normalize it

In [95]:
y = data[:, "lwage"]
y_mean = mean(y)
y_std = std(y)

y_norm = (y .- y_mean) ./ y_std


5150-element Vector{Float64}:
 -1.2402545501637507
  1.581415943901389
 -0.9952235701262617
 -0.5888284120346895
  0.6858352078834439
 -0.8916287370734166
 -0.02502721376252582
 -0.02502721376252582
 -0.8518460993196265
 -0.02502721376252582
  ⋮
  0.25770849799681306
 -0.2591347787012701
  0.3661886075001642
 -0.6009613493971904
 -0.48781225590462607
  0.29461938662075393
  1.1902001226386991
  0.9199427728221882
 -0.20974552813450043

2.2 Generate the array for the predictors $X$ (without intercept) and normalize its columns

In [96]:
X = select(data, Not([:rownames, :wage, :lwage]))
for (i, name) in enumerate(names(X))
    println("Index: $i, Name: $name")
end

Index: 1, Name: sex
Index: 2, Name: shs
Index: 3, Name: hsg
Index: 4, Name: scl
Index: 5, Name: clg
Index: 6, Name: ad
Index: 7, Name: mw
Index: 8, Name: so
Index: 9, Name: we
Index: 10, Name: ne
Index: 11, Name: exp1
Index: 12, Name: exp2
Index: 13, Name: exp3
Index: 14, Name: exp4
Index: 15, Name: occ
Index: 16, Name: occ2
Index: 17, Name: ind
Index: 18, Name: ind2


In [97]:
# Columns are continuous variables (by their indices) --> Only continuos variables should be normalized
cont_cols = [11, 12, 13, 14, 15, 17]

X_norm = copy(X)

# Only normalize continuous variables
for col in 1:size(X_norm, 2)
    if col in cont_cols
        X_norm[!, col] = (X_norm[!, col] .- mean(X_norm[!, col])) ./ std(X_norm[!, col])
    end
end

describe(X_norm)


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,sex,0.444466,0.0,0.0,1.0,0,Float64
2,shs,0.023301,0.0,0.0,1.0,0,Float64
3,hsg,0.243883,0.0,0.0,1.0,0,Float64
4,scl,0.278058,0.0,0.0,1.0,0,Float64
5,clg,0.31767,0.0,0.0,1.0,0,Float64
6,ad,0.137087,0.0,0.0,1.0,0,Float64
7,mw,0.259612,0.0,0.0,1.0,0,Float64
8,so,0.296505,0.0,0.0,1.0,0,Float64
9,we,0.216117,0.0,0.0,1.0,0,Float64


3. Split between training and testing sample. The testing sample should be 10% of the total

In [98]:
train_sample = rand(Float64, size(data)[1]) .< 0.9
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_train, X_test = X_eflex[train_sample, :], X_eflex[test_sample, :];

# 2. Creating the Lasso Cross-Validation Procedure

4. Program a function that generates a logarithmically spaced grid. The input arguments should be the natural logarithms of the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for $λ$ values to try during cross-validation.

In [99]:
function log_space_grid(log_lower_bound::Float64, log_upper_bound::Float64, log_spacing::Float64)
    grid = exp.(log_lower_bound:log_spacing:log_upper_bound)
    return grid
end


log_space_grid (generic function with 1 method)

5. Program a function to generate $k$ folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of $k$ 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

In [100]:
function gen_kfolds(X::AbstractArray, k::Int)
    n = size(X,1)

    indices = shuffle(1:n)

    folds = []
    fold_size = Int(floor(n / k))
    remainder = n % k

    start = 1
    for i in 1:k
        extra = i <= remainder ? 1 : 0  # Some folds will have 1 extra row if remainder > 0
        # Ensure we don't go out of bounds
        stop = min(start + fold_size + extra - 1, n)  # Make sure 'stop' is within bounds
        fold_indices = indices[start:stop]
        start += fold_size + extra

        # Create a boolean array to mark which rows belong to the current fold
        fold = falses(n)
        fold[fold_indices] .= true
        push!(folds, fold)
    end

    return folds
end


gen_kfolds (generic function with 1 method)

6. Program a function that integrates those that you programmed in the last two items to find the value of λ that minimizes the testing mean square error across folds. It should take the following inputs:

- Y: an array for the outcome variable.
- X: an array of predictors.
- lambda_bounds: the lower and upper bounds of the grid of lambda values.
- k: number of folds...



In [120]:
using GLMNet, Statistics

function lasso_cv(X::AbstractArray, y::AbstractArray, lambda_bounds::Tuple{Float64, Float64}, k::Int)
    # Generate a log-spaced grid of lambdas
    log_lower_bound, log_upper_bound = lambda_bounds
    lambdas = log_space_grid(log_lower_bound, log_upper_bound, 0.1)  

  
    folds = gen_kfolds(X, k)
    
    n_lambdas = length(lambdas)
    mse_per_lambda = zeros(n_lambdas)
    
    for (j, lamda) in enumerate(lambdas)
        mse_fold = 0.0  # Collect MSE for each lambda
        
        for fold in folds
            X_train = X[.!fold, :]
            y_train = y[.!fold]
            X_test = X[fold, :]
            y_test = y[fold]
            
            # Lasso model
            model = glmnet(X_train, y_train, alpha=1.0, lambda=[lamda]) 

            # Make predictions on the test set
            y_pred = predict(model, X_test)
            
            # Calculate MSE
            mse_fold += mean((y_pred .- y_test).^2)
        end
        
        # Store average MSE for the current lambda across all folds
        mse_per_lambda[j] = mse_fold / k
    end

    # Find optimal lambda with minimum MSE
    optimal_lambda_idx = argmin(mse_per_lambda)
    optimal_lambda = lambdas[optimal_lambda_idx]

    
    final_model = glmnet(X, y, alpha=1.0, lambda=[optimal_lambda])
    optimal_coef = coef(final_model)

      # Return dictionary with results
      return Dict(
        "optimal_lambda" => optimal_lambda,
        "optimal_coef" => optimal_coef,
        "all_lambdas" => lambdas,
        "all_mse" => mse_per_lambda
    )
end



lasso_cv (generic function with 1 method)

7. Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs

- optimal_model: A dictionary with the values outputed by the function defined for the previous point.
- X: an array of predictors.
- The output should be an array of predicted values.

In [121]:
function predict_outcome(optimal_model::Dict, X::AbstractArray)
    
    optimal_coef = optimal_model["optimal_coef"]
    
    y_pred = X * optimal_coef
    
    return y_pred
end


predict_outcome (generic function with 1 method)

# 3.Applying the Lasso Cross-Validation Procedure

8. Fit a simple OLS model with the training sample.

In [103]:
ols = lm(X_train, y_train)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────────────────────
             Coef.   Std. Error       t  Pr(>|t|)     Lower 95%     Upper 95%
─────────────────────────────────────────────────────────────────────────────
x1     3.80663        1.32191      2.88    0.0040    1.21492       6.39834
x2    -0.0577694      0.0169211   -3.41    0.0006   -0.0909446    -0.0245943
x3    -0.322783       0.284661    -1.13    0.2569   -0.880882      0.235316
x4     3.1786         2.16304      1.47    0.1418   -1.06219       7.41939
x5    -1.11419        0.671859    -1.66    0.0973   -2.43142       0.203038
x6     0.129175       0.0727755    1.77    0.0760   -0.0135066     0.271857
x7    -0.993105       1.27095     -0.78    0.4346   -3.48489       1.49868
x8    -0.884451       1.26523     -0.70    0.4846   -3.36502       1.59612
x9    -0.493442   

In [123]:
# Define the bounds for λ based on the natural log
lambda_bounds = (exp(-7), exp(7))  # Bounds for lambda
k = 5  # Number of folds for cross-validation

# Call the lasso_cross_validation function
optimal_model = lasso_cross_validation(y_train, X_train, lambda_bounds, k)

# Extract and print the optimal lambda and coefficients
println("Optimal Lambda: ", optimal_model["optimal_lambda"])
println("Optimal Coefficients: ", optimal_model["optimal_coef"])

UndefVarError: UndefVarError: `coef` not defined